## mount drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt

import zipfile

import tensorflow as tf
#import tensorflow_hub as hub
import math
import itertools
from sklearn.metrics import classification_report, confusion_matrix
import shutil
from keras.regularizers import l2


In [ ]:
os.chdir("/content")

# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/MyDrive/Currency_Data/Egyptian_Currency.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()
    
shutil.move("/content/content/Egyptian_Currency","/content/")

In [ ]:
shutil.move("/content/content/Egyptian_Currency","/content/")

'/content/Egyptian_Currency'

##Model

In [ ]:
base_dir = '/content/Egyptian_Currency'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')

batch_size = 128
epochs = 64
IMG_SHAPE = 400 # Our training data will consists of images with width of 400 pixels and height of 400 pixels


In [ ]:
pre_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=(400, 400, 3),classes=12)
pre_model.summary()



83689472/83683744 [==============================] - 1s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400, 400, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 199, 199, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 199, 199, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 199, 199, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [ ]:
for layer in pre_model.layers[:-17]:
    layer.trainable = False


In [ ]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255, 
                    rotation_range=15, 
                    width_shift_range=.1, 
                    height_shift_range=.1, 
                    horizontal_flip=False,
                    vertical_flip=True 
                    ,zoom_range=0.5
                    )

validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255)
 
train_data_gen = train_image_generator.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                class_mode='categorical',
                                                target_size=(IMG_SHAPE,IMG_SHAPE))

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size, 
                                                              directory=validation_dir, 
                                                              class_mode='categorical',
                                                              target_size=(IMG_SHAPE,IMG_SHAPE),shuffle=True)

In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())
model_fine.add(tf.keras.layers.Dropout(0.6))
model_fine.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 13, 13, 2048)      20861480  
_________________________________________________________________
layer_normalization (LayerNo (None, 13, 13, 2048)      26        
_________________________________________________________________
flatten (Flatten)            (None, 346112)            0         
_________________________________________________________________
dropout (Dropout)            (None, 346112)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                22151232  
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3

In [ ]:
learning_rates = [0.001,0.0003,0.0001,0.00003,0.000003]
for lr in learning_rates:
  model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  history = model_fine.fit(train_data_gen, validation_data=val_data_gen, epochs=10, steps_per_epoch=20, validation_steps=20)
  print ("\t\t\t******************* \t\t",lr," \t\t******************* \t\t \n \n")

In [ ]:
 #save model 
model_fine.save("curr_classification.h5")


In [ ]:
#Evaluating model
model_fine.evaluate_generator(val_data_gen)

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(40)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
train_data_gen.class_indices

{'.ipynb_checkpoints': 0,
 '100B': 1,
 '100F': 2,
 '10B': 3,
 '10F': 4,
 '200B': 5,
 '200F': 6,
 '20B': 7,
 '20F': 8,
 '50B': 9,
 '50F': 10,
 '5B': 11,
 '5F': 12}

In [ ]:
#load model
model=tf.keras.models.load_model("/content/drive/My Drive/curr_classification.h5")

In [ ]:
import cv2
import tensorflow as tf
y = ['100',  '100', '10',  '10',  '200',  '200',  '20', '20',  '50',  '50',  '5', '5']
#import model
model=tf.keras.models.load_model("/content/drive/My Drive/curr_classification.h5",compile=False)

def Curr_Pred(path):
    img=cv2.imread(path)
    dim=(400,400)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]

    if max(pred_probab) < 0.45:
        print("Please take another picture")
    else:    
        pred_class = list(pred_probab).index(max(pred_probab))
        return y[list(pred_probab).index(max(pred_probab))]


#predection
predec = Curr_Pred("/content/drive/My Drive/valid/10B/1 (2).jpg")        

array([2.5789283e-02, 1.9854952e-03, 4.8894208e-04, 1.6032040e-02,
       1.9381220e-02, 5.6533422e-03, 1.5688317e-01, 7.5678986e-01,
       1.0659717e-02, 4.1324091e-03, 4.8998510e-04, 1.7144473e-03],
      dtype=float32)

In [ ]:
Curr_Pred("/content/wrong_predectionXXyy213_3.jpg")

'100'

# get the accurcy of our model

in our data, we considered front and back sides of the currency as diffrent classes. but practically, if we give the model back side and it predicted it as front, that woildn't be a problem.

so we need a script to get the practical accurcy:

In [ ]:
import glob
import cv2
pred_true=0
x=['5B','10B','20B','50B','100B','200B','5F','10F','20F','50F','100F','200F']
for xx in x:
    data_path = os.path.join("/content/Curr_data_13/valid/"+xx,'*g')
    files = glob.glob(data_path)
    for f1,img in enumerate(files):
        pred_class=validate_test(img)
        if (xx == '5B' or xx == '5F') and (pred_class  == 10 or pred_class == 11):
            pred_true+= 1

        elif (xx == '10B' or xx == '10F')  and (pred_class  == 2 or pred_class == 3):
            pred_true+= 1

        elif (xx == '20B' or xx == '20F')  and (pred_class  == 6 or pred_class == 7):
            pred_true+= 1

        elif (xx == '100B'  or xx == '100F')and (pred_class  == 0 or pred_class == 1):
            pred_true+= 1

        elif (xx == '200B'  or xx == '200F')and (pred_class  == 4 or pred_class == 5):
            pred_true+= 1
        elif (xx == '50B' or xx == '50F')  and (pred_class  == 8 or pred_class == 9):
            pred_true+= 1

        else:
            img=cv2.imread(img)
            cv2.imwrite('/content/Mis_predicted3/'+str(f1)+'_'+str(pred_class)+'.jpg', img)

        if f1 % 500 ==0:
          print(pred_true)            

print(pred_true)

no_curr=0


In [ ]:
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Generator for our validation data

val_data_gen = validation_image_generator.flow_from_directory(directory="/content/drive/My Drive/valid")


Found 5600 images belonging to 12 classes.


##check models

In [ ]:

import cv2
dim=(224,224)

def validate_test(path):
    img=cv2.imread(str(path)) #str(path)
    dim=(224,224)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]
    return pred_probab



In [ ]:
import cv2

def validate_test(path):
    dim=(400,400)
    img=cv2.imread(str(path)) #str(path)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]
    return pred_probab
pred_false_true=0
very_false=0
pred_true=0
Models= ['/content/drive/My Drive/hanycurr_84.30%.h5']

n_pred_true=[]

for mod_path in Models:
    pred_true=0
    pred_false=0
    print(mod_path, '\n')
    model=tf.keras.models.load_model(mod_path,compile=False)
    
    for xx in x:
        data_path = os.path.join("/content/content/drive/My Drive/valid/"+xx,'*g')
        files = glob.glob(data_path)
        for f1,img in enumerate(files):
            pred_probab=validate_test(img)
            pred_class = list(pred_probab).index(max(pred_probab))

            if (xx == '5B' or xx == '5F') and (pred_class  == 10 or pred_class == 11) : #and max(pred_probab) > 0.5 
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1
            elif (xx == '10B' or xx == '10F')  and (pred_class  == 2 or pred_class == 3):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1
                

            elif (xx == '20B' or xx == '20F')  and (pred_class  == 6 or pred_class == 7):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '100B'  or xx == '100F')and (pred_class  == 0 or pred_class == 1):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '200B'  or xx == '200F')and (pred_class  == 4 or pred_class == 5):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '50B' or xx == '50F')  and (pred_class  == 8 or pred_class == 9):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            else:

                if max(pred_probab) > 0.5:
                    pred_false+=1
                    
                else:
                   very_false+=1
                


        print(len(files),"\t",pred_true,"\t",pred_false_true,"\t",pred_false,"\t",very_false,)                
    n_pred_true.append(pred_true)
    print(n_pred_true, pred_false)
        


/content/drive/My Drive/hanycurr_84.30%.h5 

444 	 429 	 424 	 11 	 4
431 	 835 	 827 	 30 	 10
335 	 1126 	 1111 	 63 	 21
413 	 1479 	 1448 	 106 	 38
430 	 1876 	 1841 	 136 	 41
511 	 2363 	 2319 	 149 	 52
492 	 2791 	 2736 	 195 	 70
440 	 3193 	 3127 	 217 	 86
372 	 3524 	 3441 	 242 	 102
445 	 3864 	 3762 	 316 	 133
467 	 4285 	 4170 	 339 	 156
526 	 4752 	 4625 	 377 	 177
[4752] 377
